In [ ]:
import cv2
import numpy as np

reference_image = cv2.imread('reference.jpeg', cv2.IMREAD_GRAYSCALE)
sift = cv2.SIFT_create()
keypoints_ref, descriptors_ref = sift.detectAndCompute(reference_image, None)

cap = cv2.VideoCapture(r'E:\Semester 7\Computer Vision\Lab 7\vid.mp4')

bf = cv2.BFMatcher()

search_region = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    roi = gray_frame if search_region is None else gray_frame[search_region[1]:search_region[3], search_region[0]:search_region[2]]

    keypoints_frame, descriptors_frame = sift.detectAndCompute(roi, None)

    matches = bf.knnMatch(descriptors_ref, descriptors_frame, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    if len(good_matches) > 10:
        src_pts = np.float32([keypoints_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        if search_region:
            dst_pts += [search_region[0], search_region[1]]

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if M is not None:
            h, w = reference_image.shape
            pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts, M)
            frame = cv2.polylines(frame, [np.int32(dst)], True, (0, 255, 0), 3)

            search_region = (
                max(int(dst[:, 0, 0].min() - w * 0.1), 0),
                max(int(dst[:, 0, 1].min() - h * 0.1), 0),
                min(int(dst[:, 0, 0].max() + w * 0.1), frame.shape[1]),
                min(int(dst[:, 0, 1].max() + h * 0.1), frame.shape[0])
            )
        else:
            search_region = None
    else:
        search_region = None

    cv2.imshow('Tracked Object', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
